In [ ]:
import sys
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

# **Uploading files from machine**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving meOne.jpg to meOne.jpg
Saving maOne.jpg to maOne.jpg


Using tensorfow version 1 in order to match compatibility

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
#import tensorflow as tf

tf.enable_eager_execution()


.

In [ ]:
tf.executing_eagerly()

True

# Importing required libraries

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd

%matplotlib inline
%load_ext autoreload
%autoreload 2

#np.set_printoptions(threshold=np.nan)

......

......

......

Importing fr_utils and the inception_block_v2 architecture which has already been imported from the machine to Colab

In [ ]:

from fr_utils import *
from inception_blocks_v2 import *


......

.......

.......

# 1st Model (Inception V2)

Model summary

In [ ]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))
FRmodel.summary()

Model: "FaceRecoModel"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3, 96, 96)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 3, 102, 102)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 48, 48)   9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 64, 48, 48)   256         conv1[0][0]                      
______________________________________________________________________________________

In [ ]:
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


....

...

....

.......

# TRIPLET LOSS

This is the loss function that will preferably be used to train the model in case the model needs to be trained at all to fit the requirement of the user

In [ ]:

def triplet_loss(y_true, y_pred, alpha = 0.2):     # alpha taken as 0.2
  
  anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
  
  #print(anchor.shape)
  
  pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis = -1)
  
  neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis = -1)
  
  basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
  
  loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0), axis = None)
  
  return loss

.....

....

...

....

....

## **COMPILING THE MODEL**

Only compiling model 1 (Inceptionv2) as of now

Loading pretrained weights into the model skeleton. The weights are pretrained on the imageNet dataset

In [ ]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [ ]:
# done for the day : 15.09.2020

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


In [ ]:
with  tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(loss.eval()))

In [ ]:
tf.executing_eagerly()

.......

# Creating the dataset

In [ ]:
database = {}
database["samarpan"] = img_to_encoding("samarpan.jpg", FRmodel)
database["sampit"] = img_to_encoding("sampit.jpg", FRmodel)


In [ ]:
print(len(database))

In [ ]:
tf.executing_eagerly()

True

.....

.......

.......

.......

.....

# **FACE VERIFICATION METHOD**

In [ ]:
def faceVerification(image_path, key, database, model):
  
  encoding = img_to_encoding(image_path, model)
  
  dist = np.linalg.norm(encoding - database[key])
  
  if dist < 0.7:
    print("It's " + str(key) + ", welcome in!")
    door_open = True
    
  else:
    print("It's not " + str(key) + ", please go away")
    door_open = False
        
        
    return dist, door_open

.....

# Face Verification Operation

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
faceVerification("samarpanTester.jpg", "samarpan", database, FRmodel)

.......

.......

.......

........

# **FACE RECOGNITION METHOD**

In [ ]:
def faceRecognition(image_path, database, model):
  
  encoding = img_to_encoding(image_path, model)
  min_dist = 999
  
  for (name, db_enc) in database.items():
    dist = np.linalg.norm(encoding - db_enc)
    
    if dist < min_dist:
      min_dist = dist
      identity = name

    
    
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

.......

#**Face Recognition Operation**

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
faceRecognition("meTest.jpg", database, FRmodel)